In [ ]:
import pyodbc

conn_str = (
    "Driver = {ODBC Driver 17 for SQL Server};" #Dricer sürümü bizim için önemli. 17 sayısı farazi yazıldı.
    "Server = tcp:kendi-synapse-server.database.windows.net;"
    "Database = kendi-db;"
    "Uid = "
    "Pwd = "
    "Encrypt = yes;"
    "TrustServerCertificate = no;"
    "Connection Timeout = 30;"
)

#bağlantı açma

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

#test sorgusu 

cursor.execute("SELECT TOP 5 sale_id, vehicle_vin, price_usd FROM core_sales;")
for row in cursor.fetchall():
    print(row)

In [ ]:
import pandas as pd

df = pd.read_csv("smandira_sales.csv")

 #audit verisi ekleme
df["load_id"] =    
df["ingested_at"] =    

In [ ]:
df["vehicle_vin"] = df["vehicle_vin"].str.upper()

fx_rate = 41.33
df["price_usd"] = df.apply(
    lambda x : ["price"] * fx_rate if x["currency"] == "USD" else x["price"],

)

df = df[df["price_usd"] < 500000]

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    'mssql+pyodbcÇ//username:......'
)

#dataframe'i staging tablosuna yazma


df.to_sql(
    'staging_sales',
    con=engine,
    if_exists='append',
    index=False,
    method='multi'
)

#büyük batchlerde fast_executemany = True --> 10 kat hızlandırır

Streaming + Batch Hybrid ETL

In [ ]:
#Satış verileri günlük CSV geliyor

#Servis verileri IoT cihazlardan geliyor

#ETL süreci 2 kanaldan işler

#1-> Batch = CSV --> staging --> core --> mart
#2 -> Kafka --> Synapse -->core_service tablosu --> mart

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

#Extract
df = pd.read_csv("../data/samandira_sales.csv")

#Transform
df["vehicle_vin"] = df["vehicle_vin"].str.upper()

fx_rate = 41.33
df["price_usd"] = df.apply(
    lambda x : ["price"] * fx_rate if x["currency"] == "USD" else x["price"],

)

df = df[df["price_usd"] < 500000]

#Load
engine = create_engine(
    'mssql+pyodbcÇ//username:......'
)

#dataframe'i staging tablosuna yazma
df.to_sql(
    'staging_sales',
    con=engine,
    if_exists='append',
    index=False,
    method='multi'
)

In [ ]:
from kafka import KafkaConsumer
import json, pyodbc

consumer = KafkaConsumer("service-events", bootstrap_servers = "broker:9092")
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

for x in consumer:
    event = json.load(x.value)
    cursor.execute("MERGE INTO core_service t" \
    "USING (SELECT ? AS vin, ? AS part, ? AS cost, ? AS service_time) s" \
    "ON t.vin = s.vin AND t.sevice_time = s.service_time " \
    "WHEN MATCHED THEN UPDATE SET cost = s.cost" \
    "WHEN NOT MATCH THEN INSERT (vin, part, service_time)")